In [1]:
#https://newsdata.io/api-key
import requests

key = "pub_74016650639a414b2f59c443dc14602226fb"
resp = requests.get('https://newsdata.io/api/1/news?apikey=pub_74016650639a414b2f59c443dc14602226fb&q=car')
print(resp.json())

{'status': 'success', 'totalResults': 659, 'results': [{'title': 'Josh Moffett makes it four from four after Carlow Rally victory', 'link': 'https://highlandradio.com/2022/05/15/josh-moffett-makes-it-four-from-four-after-carlow-rally-victory/', 'keywords': ['Sport'], 'creator': ['highlandsport'], 'video_url': None, 'description': 'Monaghan’s Josh Moffett (Hyundai i20 R5) maintained his dominance of the Triton Showers-backed Motorsport Ireland National Rally Championship courtesy of an emphatic victory in the Sheehy Motor Group Carlow Rally that marked the halfway point of the eight round series. It was Moffett’s fourth straight win of the national campaign. Co-driven by Limerick’s Keith Moriarty, he finished … Josh Moffett makes it four from four after Carlow Rally victory Read More » The post Josh Moffett makes it four from four after Carlow Rally victory appeared first on Highland Radio - Latest Donegal News and Sport.', 'content': 'Monaghan’s Josh Moffett (right) and Limerick’s Keit

In [2]:
results = resp.json()['results']

In [3]:
results

[{'title': 'Josh Moffett makes it four from four after Carlow Rally victory',
  'link': 'https://highlandradio.com/2022/05/15/josh-moffett-makes-it-four-from-four-after-carlow-rally-victory/',
  'keywords': ['Sport'],
  'creator': ['highlandsport'],
  'video_url': None,
  'description': 'Monaghan’s Josh Moffett (Hyundai i20 R5) maintained his dominance of the Triton Showers-backed Motorsport Ireland National Rally Championship courtesy of an emphatic victory in the Sheehy Motor Group Carlow Rally that marked the halfway point of the eight round series. It was Moffett’s fourth straight win of the national campaign. Co-driven by Limerick’s Keith Moriarty, he finished … Josh Moffett makes it four from four after Carlow Rally victory Read More » The post Josh Moffett makes it four from four after Carlow Rally victory appeared first on Highland Radio - Latest Donegal News and Sport.',
  'content': 'Monaghan’s Josh Moffett (right) and Limerick’s Keith Moriarty pictured at the finish ramp aft

In [36]:
import pandas as pd

In [37]:
df = pd.DataFrame(results)
df.index =  pd.to_datetime(df['pubDate'])
df = df.drop(columns='pubDate')
df = df.sort_values(by='pubDate')

In [48]:
import numpy as np
from numpy.dual import svd
from numpy import unicode, asarray, sqrt, log


class LSI(object):
	def __init__(self, stopwords, ignorechars, docs):
		self.wdict = {}
		self.dictionary = []
		self.stopwords = stopwords
		self.docs = []
		if type(ignorechars) == unicode: ignorechars = ignorechars.encode('utf-8')
		self.ignorechars = ignorechars
		for doc in docs: self.add_doc(doc)

	def prepare(self):
		self.build()
		self.calc()

	def dic(self, word, add = False):
		if type(word) == unicode: word = word.encode('utf-8')
		word = word.lower().translate(None, self.ignorechars)
		word = word.decode('utf-8')
		import nltk
		stemmer = nltk.PorterStemmer()
		word = stemmer.stem(word)
		if word in self.dictionary: return self.dictionary.index(word)
		else:
			if add:
				self.dictionary.append(word)
				return len(self.dictionary) - 1
			else: return None

	def add_doc(self, doc):
		try:
			words = [self.dic(word, True) for word in doc.lower().split()]
			self.docs.append(words)
			for word in words:
				if word in self.stopwords:  continue
				elif word in self.wdict:   self.wdict[word].append(len(self.docs) - 1)
				else:                      self.wdict[word] = [len(self.docs) - 1]
		except:
			pass

	def build(self):
		self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 0]
		self.keys.sort()
		self.A = np.zeros([len(self.keys), len(self.docs)])
		for i, k in enumerate(self.keys):
			for d in self.wdict[k]:
				self.A[i,d] += 1

	def calc(self):
		self.U, self.S, self.Vt = svd(self.A)

	def TFIDF(self):
		wordsPerDoc = sum(self.A, axis=0)
		docsPerWord = sum(asarray(self.A > 0, 'i'), axis=1)
		rows, cols = self.A.shape
		for i in range(rows):
			for j in range(cols):
				self.A[i,j] = (self.A[i,j] / wordsPerDoc[j]) * log(float(cols) / docsPerWord[i])

	def dump_src(self):
		self.prepare()
		print ('Здесь представлен расчет матрицы ')
		for i, row in enumerate(self.A):
			print (self.dictionary[i], row)

	def print_svd(self):
		self.prepare()
		print ('Здесь сингулярные значения')
		print (self.S)
		print ('Здесь первые 3 колонки U матрица ')
		for i, row in enumerate(self.U):
			print (self.dictionary[self.keys[i]], row[0:3])
		print ('Здесь первые 3 строчки Vt матрица')
		print -1*self.Vt[0:3, :]

	def find(self, word):
		self.prepare()
		idx = self.dic(word)
		if not idx:
			print ('слово не встерчается')
			return []
		if not idx in self.keys:
			print ('слово отброшено как не имеющее значения которое через stopwords')
			return []
		idx = self.keys.index(idx)
		print ('word --- ', word, '=', self.dictionary[self.keys[idx]], '.\n')
		# получаем координаты слова
		wx, wy = (-1 * self.U[:, 1:3])[idx]
		print('word {}\t{:0.2f}\t{:0.2f}\t{}\n'.format(idx, wx, wy, word))
		arts = []
		xx, yy = -1 * self.Vt[1:3, :]
		for k, v in enumerate(self.docs):
			ax, ay = xx[k], yy[k]
			dx, dy = float(wx - ax), float(wy - ay)
			arts.append((k, v, ax, ay, sqrt(dx * dx + dy * dy)))
		return sorted(arts, key = lambda a: a[4])

/tmp/ipykernel_7382/2218928967.py:3: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import unicode, asarray, sqrt, log


In [49]:
containable = [x for x in df['content']]
containable


print(containable)

['Former Raiders receiver Henry Ruggs III continues to face multiple felony charges following a car accident that claimed the life of 23-year-old Tina Tintor last November. And the legal case, as many legal cases do, continues to move slowly through the justice system. Most recently, via the Associated Press, a judge has ruled that prosecutors can obtain medical records regarding injuries sustained by Ruggs’s girlfriend, Kiara Je’nai Kilgo-Washington. Prosecutors claim that they want the medical records in order to prove that Kilgo-Washington suffered substantial injuries in the crash, since those injuries provide a separate basis for charges against Ruggs. The prosecution also may be looking to seal off grounds for reasonable doubt based on whether Ruggs was actually driving the car. If he elects not to testify and if Kilgo-Washington invokes the Fifth Amendment when asked whether she was driving, can it be proven to the satisfaction of the very high legal standard applicable to crimi

*settings*

In [50]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import nltk
from nltk import *
from nltk.corpus import brown
stopwords= nltk.corpus.stopwords.words('english')
docs = containable
stem='english'
print(containable)

['Former Raiders receiver Henry Ruggs III continues to face multiple felony charges following a car accident that claimed the life of 23-year-old Tina Tintor last November. And the legal case, as many legal cases do, continues to move slowly through the justice system. Most recently, via the Associated Press, a judge has ruled that prosecutors can obtain medical records regarding injuries sustained by Ruggs’s girlfriend, Kiara Je’nai Kilgo-Washington. Prosecutors claim that they want the medical records in order to prove that Kilgo-Washington suffered substantial injuries in the crash, since those injuries provide a separate basis for charges against Ruggs. The prosecution also may be looking to seal off grounds for reasonable doubt based on whether Ruggs was actually driving the car. If he elects not to testify and if Kilgo-Washington invokes the Fifth Amendment when asked whether she was driving, can it be proven to the satisfaction of the very high legal standard applicable to crimi

In [51]:
ignorechars = ''',:'!'''
word = "cars"
lsa = LSI([], ignorechars, docs)
lsa.build()
lsa.dump_src()
lsa.calc()
lsa.print_svd()

for res in lsa.find(word):
	print(res[0], res[4], res[1], docs[res[0]])

Здесь представлен расчет матрицы 
former [1. 0. 0. 2.]
raider [1. 0. 0. 0.]
receiv [1. 1. 0. 1.]
henri [1. 0. 0. 0.]
rugg [7. 0. 0. 0.]
iii [1. 0. 0. 0.]
continu [2. 0. 0. 0.]
to [15. 25.  0. 12.]
face [1. 0. 0. 0.]
multipl [1. 0. 0. 0.]
feloni [1. 0. 0. 0.]
charg [3. 0. 0. 0.]
follow [1. 0. 0. 0.]
a [ 6. 12. 10. 20.]
car [2. 1. 0. 2.]
accid [1. 0. 0. 0.]
that [14. 10.  0.  5.]
claim [2. 0. 0. 1.]
the [26. 18.  0. 67.]
life [1. 0. 0. 0.]
of [ 9. 13.  0. 31.]
23-year-old [1. 0. 0. 0.]
tina [1. 0. 0. 0.]
tintor [1. 0. 0. 0.]
last [1. 0. 0. 2.]
november. [1. 0. 0. 0.]
and [ 3. 20.  0. 28.]
legal [3. 0. 0. 0.]
case [3. 0. 0. 0.]
as [1. 7. 0. 2.]
mani [1. 0. 0. 1.]
do [1. 3. 0. 1.]
move [1. 0. 0. 2.]
slowli [1. 0. 0. 0.]
through [1. 1. 0. 2.]
justic [2. 0. 0. 0.]
system. [1. 0. 0. 0.]
most [1. 2. 0. 1.]
recent [1. 0. 0. 0.]
via [1. 0. 0. 0.]
associ [1. 0. 0. 0.]
press [1. 0. 0. 0.]
judg [1. 0. 0. 0.]
ha [1. 0. 6. 0.]
rule [1. 0. 0. 0.]
prosecutor [4. 0. 0. 0.]
can [2. 0. 0. 0.]
obtain [1. 0

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'float'